In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn as nn
from tqdm.notebook import tqdm
from pytorch_lightning import LightningModule, Trainer

## Generate data

In [16]:
N_data = 50000

data_x = np.linspace(1.0, 20.0, N_data)[:, np.newaxis]
data_y = (
    0.02 * data_x
    + 0.3 * np.sin(data_x)
    + 1e-1 * np.log(data_x) ** 2
    + 0.2 * np.random.randn(N_data, 1)
)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data_x, data_y, test_size=0.2, random_state=42
)
X_train, X_test, y_train, y_test = (
    torch.FloatTensor(X_train),
    torch.FloatTensor(X_test),
    torch.FloatTensor(y_train),
    torch.FloatTensor(y_test),
)

## Previous code

In [23]:
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

# create the dataloaders
train_dl = DataLoader(train_ds, batch_size=250, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=5000, shuffle=False)

# the model
class TempNNBigger(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1, 100)
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 100)
        self.layer4 = nn.Linear(100, 100)
        self.layer5 = nn.Linear(100, 1)

    def forward(self, x):
        out = self.layer1(x)
        out = torch.nn.ReLU()(out)
        out = self.layer2(out)
        out = torch.nn.ReLU()(out)
        out = self.layer3(out)
        out = torch.nn.ReLU()(out)
        out = self.layer4(out)
        out = torch.nn.ReLU()(out)
        out = self.layer5(out)
        return out


model = TempNNBigger()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


def train_one_epoch(net, optimizer, loss_func, dataloader):
    batch_losses = []
    net.train()
    for x, y in dataloader:
        optimizer.zero_grad()
        output = net(x)
        loss = loss_func(output, y)
        batch_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.mean(batch_losses)


def test_loss(net, loss_func, dataloader):
    val_losses = []
    net.eval()
    for x, y in dataloader:
        output = net(x)
        loss = loss_func(output, y)
        val_losses.append(loss.item())
    return np.mean(val_losses)


losses = []
# training loop
for ep in tqdm(range(20)):
    losses.append(
        [
            train_one_epoch(model, optimizer, nn.MSELoss(), train_dl),
            test_loss(model, nn.MSELoss(), test_dl),
        ]
    )

  0%|          | 0/20 [00:00<?, ?it/s]

## Pytorch Lightning

In [24]:
class LitModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.net = TempNNBigger()
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        # Define forward pass
        out = self.net(x)

        return out

    def training_step(self, batch, batch_idx):
        # Define training step
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)

        return loss

    def configure_optimizers(self):
        # Configure optimizer
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def train_dataloader(self):
        # Setup train dataloader
        dataset = TensorDataset(X_train, y_train)
        loader = DataLoader(dataset, batch_size=250, shuffle=True, num_workers=0)
        return loader

    def val_dataloader(self):
        # Setup validation dataloader
        dataset = TensorDataset(X_test, y_test)
        loader = DataLoader(dataset, batch_size=5000, num_workers=0)
        return loader

    def validation_step(self, batch, batch_idx):
        # Define validation step
        x, y = batch
        y_hat = self(x)
        return {"val_loss": self.loss_fn(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # Define validation epoch end
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        return {"avg_val_loss": avg_loss}

In [25]:
model = LitModel()

In [26]:
trainer = Trainer(max_epochs=20, accelerator="mps", devices=1)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
trainer.fit(model)


  | Name    | Type         | Params
-----------------------------------------
0 | net     | TempNNBigger | 30.6 K
1 | loss_fn | MSELoss      | 0     
-----------------------------------------
30.6 K    Trainable params
0         Non-trainable params
30.6 K    Total params
0.122     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
